In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
train = pd.read_csv("loan_train.csv").dropna(how='all',axis=1)
test = pd.read_csv("loan_test.csv").dropna(how='all',axis=1)
train = train.drop(train[train.loan_status == "Current"].index).reset_index(drop=True)
test = test.drop(test[test.loan_status == "Current"].index).reset_index(drop=True)

In [2]:
train.drop('id', axis=1, inplace=True)
train.drop('member_id', axis=1, inplace=True)
train['term'] = train['term'].astype('category').cat.codes
train.drop('int_rate',axis=1,inplace=True)

train['grade'] = train['grade'].astype('category').cat.codes
train['sub_grade'] = (train['sub_grade'].astype('category').cat.codes%5)/5
train['grade'] += train['sub_grade']
train.drop('sub_grade',axis=1,inplace=True)

train.drop('emp_title', axis=1, inplace=True)
train.drop('emp_length', axis=1, inplace=True)

train['home_ownership'].replace(['MORTGAGE','OTHER','RENT'],0,inplace=True)
train['home_ownership'].replace('OWN',1,inplace=True)
train['home_ownership'] = train['home_ownership'].astype('int8')

train['verification_status'].replace('Source Verified','Verified',inplace=True)
train['verification_status'] = train['verification_status'].astype('category').cat.codes

train.drop('pymnt_plan',axis=1,inplace=True)
train.drop('url',axis=1,inplace=True)
train.drop('desc',axis=1,inplace=True)

train['purpose'] = train['purpose'].astype('category').cat.codes

train.drop('title',axis=1,inplace=True)

train.drop('zip_code',axis=1,inplace=True)
train['addr_state'][0] = 'IA'
train['addr_state'][1] = 'NE'
train['addr_state'] = train['addr_state'].astype('category').cat.codes

train.drop('earliest_cr_line',axis=1,inplace=True)

train.drop(['delinq_amnt','chargeoff_within_12_mths','acc_now_delinq','policy_code','next_pymnt_d','application_type','tax_liens','collections_12_mths_ex_med','initial_list_status','revol_util','mths_since_last_delinq','mths_since_last_record','last_pymnt_amnt','last_credit_pull_d','issue_d','last_pymnt_d'], axis=1, inplace=True)
train = train[train['pub_rec_bankruptcies'].notna()].reset_index(drop=True)

train_labels = train['loan_status'].replace('Charged Off',0).replace('Fully Paid',1).astype('int8')
train.drop('loan_status',axis=1,inplace=True)

/tmp/ipykernel_188042/1306930417.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['addr_state'][0] = 'IA'
/tmp/ipykernel_188042/1306930417.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['addr_state'][1] = 'NE'


In [3]:
test.drop('id', axis=1, inplace=True)
test.drop('member_id', axis=1, inplace=True)
test['term'] = test['term'].astype('category').cat.codes
test.drop('int_rate',axis=1,inplace=True)

test['grade'] = test['grade'].astype('category').cat.codes
test['sub_grade'] = (test['sub_grade'].astype('category').cat.codes%5)/5
test['grade'] += test['sub_grade']
test.drop('sub_grade',axis=1,inplace=True)

test.drop('emp_title', axis=1, inplace=True)
test.drop('emp_length', axis=1, inplace=True)

test['home_ownership'].replace(['MORTGAGE','OTHER','RENT','NONE'],0,inplace=True)
test['home_ownership'].replace('OWN',1,inplace=True)
test['home_ownership'] = test['home_ownership'].astype('int8')

test['verification_status'].replace('Source Verified','Verified',inplace=True)
test['verification_status'] = test['verification_status'].astype('category').cat.codes

test.drop('pymnt_plan',axis=1,inplace=True)
test.drop('url',axis=1,inplace=True)
test.drop('desc',axis=1,inplace=True)

test['purpose'] = test['purpose'].astype('category').cat.codes

test.drop('title',axis=1,inplace=True)

test.drop('zip_code',axis=1,inplace=True)
test['addr_state'][0] = 'IN'
test['addr_state'] = test['addr_state'].astype('category').cat.codes

test.drop('earliest_cr_line',axis=1,inplace=True)

test.drop(['delinq_amnt','chargeoff_within_12_mths','acc_now_delinq','policy_code','next_pymnt_d','application_type','tax_liens','collections_12_mths_ex_med','initial_list_status','revol_util','mths_since_last_delinq','mths_since_last_record','last_pymnt_amnt','last_credit_pull_d','issue_d','last_pymnt_d'], axis=1, inplace=True)
test = test[test['pub_rec_bankruptcies'].notna()].reset_index(drop=True)

test_labels = test['loan_status'].replace('Charged Off',0).replace('Fully Paid',1).astype('int8')
test.drop('loan_status',axis=1,inplace=True)

/tmp/ipykernel_188042/1652932717.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['addr_state'][0] = 'IN'


In [31]:
clf = GradientBoostingClassifier(max_depth=7,n_estimators=5000)
clf.fit(train,train_labels)
pred = clf.predict(test)
print("precision =", precision_score(test_labels,pred))
print("recall =", recall_score(test_labels,pred))
print("accuracy =", accuracy_score(test_labels,pred))

precision = 0.9988219454729047
recall = 0.9985698662404308
accuracy = 0.9977850814518434


In [36]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(train,train_labels)
dt_pred = tree.predict(test)
print("precision =", precision_score(test_labels,dt_pred))
print("recall =", recall_score(test_labels,dt_pred))
print("accuracy =", accuracy_score(test_labels,dt_pred))

precision = 0.9973932055163135
recall = 0.9978127366030117
accuracy = 0.9959274078308088
